### Práctica de Formatos de datos: CSV y JSON

#### Ejercicio 1[5 puntos]
Considerar el archivo __Contaminación.csv__. El archivo contiene la información recogida por las estaciones de control de calidad del aire durante el año 2019.

Una descripción detallada de la estructura de los datos se puede encontrar en el siguiente enlace: https://datos.madrid.es/FWProjects/egob/Catalogo/MedioAmbiente/Aire/Ficheros/Interprete_ficheros_%20calidad_%20del_%20aire_global.pdf

La información contenida por columnas es la siguiente:
* __PROVINCIA__: Código numérico que representa la provincia. 
* __MUNICIPIO__: Código numérico que representa el municipio. 
* __ESTACIÓN__: Código numérico de hasta dos dígitos que representa la estación. Consultar ANEXO I de la documentación
* __MAGNITUD__: Código numérico de hasta dos dígitos que representa la magnitud.Consultar ANEXO II de la documentación
* __PUNTO MUESTREO__: Código numérico formado por 3 subcódigos. El primer código representa la estación, el segundo código representa la magnitud, y el tercer código representa la técnica de medida.
* __MES__: Mes de la toma de datos. Valor entre 1 y 12.
* __D01-D31__: Medidas tomadas desde el día D01 hasta el día D31 del mes correspondiente.
* __V01-V31__: Validez de las medidas tomadas desde el día D01 hasta el día D31 del mes correspondiente. Puede valer V(Válido), N(No válido), no se conoce(vacío) o bien otra cadena(se considera incorrecto).


Se pide crear un programa que realice las siguientes operaciones:
* Leer los datos del archivo __Contaminación.csv__
* Transformar los datos leidos de manera que aquellos valores que no sean válidos ("N") su valor sea sustituido por 0.
* Obtener y  mostrar por pantalla para cada tipo de contaminante (columna Magnitud) y para cada estación (columna ESTACIÓN), la suma total de la contaminación que se ha producido durante el año 2019, es decir:

|Estación|Contaminante1| Contaminante2| Contaminante3| ....Contaminanten|
|----|-------------|-------------|-------------|-------------|
|Estación1|Subtotal11 |--- |--- |Subtotal1n|  
|Estación2|--- |--- |--- |--- | 
|   ...|--- |--- |--- |--- |     
|Estaciónn|Subtotal1n |--- |--- |Subtotalnn| 
     
* Generar un documento __Salida.csv__ donde cada línea del documento contiene la siguiente información:
    Estación1, Contaminante1,Subtotal1, Contaminante2,Subtotal2,......................,Contaminanten,Subtotaln
    
    Estaciónm, Contaminante1,Subtotal1, Contaminante2,Subtotal2,......................,Contaminanten,Subtotaln
    
Tanto en lo que mostréis por pantalla como lo que escribáis en el archivo usad los nombres de los contaminantes y de las estaciones que aparecen en los anexos de la documentación indicada más arriba.
    

In [22]:
import csv
import re

diccionarioContaminantes = {1:"Dióxido de Azufre",6:"Monóxido de Carbono",
                 7:"Monóxido de Nitrógeno",8:"Dióxido de Nitrógeno",
                 9:"Partículas <2.5 µm",10:"Partículas < 10µm",12:"Óxidos de Nitrógeno",
                 14:"Ozono",20:"Tolueno",30:"Benceno",35:"Etilbenceno",37:"Metaxileno",
                 38:"Paraxileno",39:"Ortoxileno",42:"Hidrocarburos totales(hexano)",43:"Metano",
                44: "Hidrocarburos no metánicos (hexano)"}

diccionarioEstaciones = {1:"Pº.Recoletos",2:"Glta. de Carlos V", 35:"Pza. del Carmen", 4:"Pza. de España", 39:"Barrio del Pilar",
                    6:"Pza. Dr. Marañón", 7:"Pza. M. de Salamanca", 8:"Escuelas Aguirre", 9:"Pza. Luca de Tena", 38:"Cuatro Caminos",
                     11:"Av. Ramón y Cajal", 12:"Pza. Manuel Becerra", 40:"Vallecas",14:"Pza. Fdez. Ladreda",15:"Pza. Castilla",
                     16:"Arturo Soria",17:"Villaverde Alto",18:"C/ Farolillo",19:"Huerta Castañeda",36:"Moratalaz",21:"Pza. Cristo Rey",
                     22:"Pº. Pontones",23:"Final C/ Alcalá",24:"Casa de Campo",25:"Santa Eugenia",26:"Urb. Embajada (Barajas)",
                     27:"Barajas",47:"Méndez Álvaro",48:"Pº Castellana",49:"Retiro",50:"Pza. Castilla",54:"Ensanche Vallecas",
                     55:"Urb. Embajada (Barajas)",56:"Plaza Elíptica",57:"Sanchinarro",58:"El Pardo",59:"Parque Juan Carlos I",
                     60:"Tres Olivos"}

csvarchivo = open('Contaminación.csv',encoding="utf8",errors='ignore')
entrada = csv.reader(csvarchivo, delimiter=";")
indicesValorar = []
indicesDatos = []


def escribirConsola(tabla):
    #cabecera = "Estacion "
    #for valor in diccionarioContaminantes.values():
    #    cabecera += valor + " "
    #print(cabecera)
    for fila in tablaFinal:
        estacion = True
        datosEstacion = []
        print("Estacion: ", (diccionarioEstaciones[int(fila[0])]))
        estacion = False
        contador = 0
        for contaminante in diccionarioContaminantes.values():
            contador += 1
            print("\t", contaminante, " : ", float(fila[contador]))
        print("\n")
        
def escribirCSV(tabla):
    archivoSalida=open("Salida.csv","w",newline='')
    salidaEscritor=csv.writer(archivoSalida)
    for fila in tabla:
        listaSalida = []
        contadorFila = 1
        listaSalida.append(diccionarioEstaciones[int(fila[0])])
        for contaminante in diccionarioContaminantes.values():
            listaSalida.append(contaminante)
            listaSalida.append(fila[contadorFila])
            contadorFila+=1
        salidaEscritor.writerow(listaSalida)
    archivoSalida.close()


def parseaNoValidos(indicesValorar, fila):
    for indice in indicesValorar:
        if fila[indice]=="N":
                fila[indice-1] = 0
    return fila

def tratarDatos(i):
    
    fila = []
    contadores=[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
    indicesContaminantes=[1,6,7,8,9,10,12,14,20,30,35,37,38,39,42,43,44]
    estacion = i[0]
    
    #Para cada dato que tiene la fila se modifica el contador correspondiente
    for j in i[1]:
        contadores[indicesContaminantes.index(int(j[0]))]=j[1]
    
    
    #Se guarda la informacion de toda la linea en la variable fila
    fila.append(estacion)
    #Se recorren todos los contadores para introducir tambien aquellos que no hay muestra
    for z in contadores:
        fila.append(z)
    return fila

def acumulaDatos(contadorActual, indicesDatos, fila):
    for indice in indicesDatos:
        contadorActual += float(fila[indice])
    return contadorActual


contador1 = 0
verifica = 'V'
patron = re.compile(verifica)

magnitudes=[]
estaciones=[]

estacionActual = None
magnitudActual = None
contadorActual = 0
contadoresEstacionActual = []
tablaDatos = []
cogidos = False
for i in entrada:
    if cogidos == False:
        for j in i:
            if(patron.match(j)):
                indicesValorar.append(contador1)
                indicesDatos.append(contador1-1)
            contador1+=1
        cogidos=True
    else:
        i = parseaNoValidos(indicesValorar, i)
        if estacionActual == None:
            estacionActual = i[2]
            magnitudActual = i[3]
            contadorActual = acumulaDatos(contadorActual, indicesDatos, i)
            
        elif estacionActual != i[2]:
            contadoresEstacionActual.append([magnitudActual, contadorActual])
            tablaDatos.append([estacionActual, contadoresEstacionActual])
            estacionActual = i[2]
            magnitudActual = i[3]
            contadorActual = 0
            contadorActual = acumulaDatos(contadorActual, indicesDatos, i)
            contadoresEstacionActual = []
        else:
            if magnitudActual == i[3]:
                contadorActual = acumulaDatos(contadorActual, indicesDatos, i)

            else:
                contadoresEstacionActual.append([magnitudActual, contadorActual])
                magnitudActual = i[3]
                contadorActual = 0
                contadorActual = acumulaDatos(contadorActual, indicesDatos, i)

tablaDatos.append([estacionActual, contadoresEstacionActual])
tablaFinal = []

for i in tablaDatos:
    tablaFinal.append(tratarDatos(i))

    
escribirConsola(tablaFinal)
escribirCSV(tablaFinal)

Estacion:  Pza. de España
	 Dióxido de Azufre  :  2016.0
	 Monóxido de Carbono  :  104.99999999999984
	 Monóxido de Nitrógeno  :  6373.0
	 Dióxido de Nitrógeno  :  10392.0
	 Partículas <2.5 µm  :  0.0
	 Partículas < 10µm  :  0.0
	 Óxidos de Nitrógeno  :  20147.0
	 Ozono  :  0.0
	 Tolueno  :  0.0
	 Benceno  :  0.0
	 Etilbenceno  :  0.0
	 Metaxileno  :  0.0
	 Paraxileno  :  0.0
	 Ortoxileno  :  0.0
	 Hidrocarburos totales(hexano)  :  0.0
	 Metano  :  0.0
	 Hidrocarburos no metánicos (hexano)  :  0.0


Estacion:  Escuelas Aguirre
	 Dióxido de Azufre  :  2098.0
	 Monóxido de Carbono  :  46.700000000000145
	 Monóxido de Nitrógeno  :  4819.0
	 Dióxido de Nitrógeno  :  12109.0
	 Partículas <2.5 µm  :  2595.0
	 Partículas < 10µm  :  5230.0
	 Óxidos de Nitrógeno  :  19486.0
	 Ozono  :  12049.0
	 Tolueno  :  632.9000000000001
	 Benceno  :  146.8000000000002
	 Etilbenceno  :  113.50000000000009
	 Metaxileno  :  0.0
	 Paraxileno  :  0.0
	 Ortoxileno  :  0.0
	 Hidrocarburos totales(hexano)  :  375.

	 Paraxileno  :  0.0
	 Ortoxileno  :  0.0
	 Hidrocarburos totales(hexano)  :  0.0
	 Metano  :  0.0
	 Hidrocarburos no metánicos (hexano)  :  0.0


Estacion:  El Pardo
	 Dióxido de Azufre  :  0.0
	 Monóxido de Carbono  :  0.0
	 Monóxido de Nitrógeno  :  905.0
	 Dióxido de Nitrógeno  :  3702.0
	 Partículas <2.5 µm  :  0.0
	 Partículas < 10µm  :  0.0
	 Óxidos de Nitrógeno  :  5098.0
	 Ozono  :  16015.0
	 Tolueno  :  0.0
	 Benceno  :  0.0
	 Etilbenceno  :  0.0
	 Metaxileno  :  0.0
	 Paraxileno  :  0.0
	 Ortoxileno  :  0.0
	 Hidrocarburos totales(hexano)  :  0.0
	 Metano  :  0.0
	 Hidrocarburos no metánicos (hexano)  :  0.0


Estacion:  Parque Juan Carlos I
	 Dióxido de Azufre  :  0.0
	 Monóxido de Carbono  :  0.0
	 Monóxido de Nitrógeno  :  1803.0
	 Dióxido de Nitrógeno  :  5541.0
	 Partículas <2.5 µm  :  0.0
	 Partículas < 10µm  :  0.0
	 Óxidos de Nitrógeno  :  8298.0
	 Ozono  :  14401.0
	 Tolueno  :  0.0
	 Benceno  :  0.0
	 Etilbenceno  :  0.0
	 Metaxileno  :  0.0
	 Paraxileno  :  0.0
	 

#### Ejercicio 2[5 puntos]
Considerar los archivos  __Estaciones.csv__ y __Museos.json__ que incluye información sobre las estaciones de control de la calidad del aire y sobre los museos de Madrid respectivamente.

El objetivo de este ejercicio es utilizar la información obtenida en el ejercicio 1 de forma que para cada uno de los museos que aparece en el archivo dado realice las siguientes operaciones:
* Obtener las 3 estaciones más cercanas a cada museo.
* Obtener y mostrar por pantalla para cada museo la siguiente información donde ValorX es el subtotal del nivel de contaminación más alto de todos los contaminantes medidos en la estaciónX, y TipoContaminanteX es el tipo del contaminante cuyo valor es mostrado en la celda anterior.

|Museo|Estación1|TipoContaminante1| Estación2|TipoContaminante2| Estación3|TipoContaminante3|
|----|-------------|-------------|-------------|----|----|----|
|Museo1|Valor1 |Tipo1 |Valor2|Tipo2|Valor3|  Tipo3 |
|Museo2|--- |--- |--- |--- | --- | --- | 
|   ...|--- |--- |--- |--- | --- | --- |     
|Museon|Valor1 |Tipo1 |Valor2|Tipo2|Valor3|  Tipo3 |

* Generar un documento __Salida.json__ con la siguiente estructura:

Tanto en lo que mostréis por pantalla como lo que escribáis en el archivo usad los nombres de los contaminantes y de las estaciones que aparecen en los anexos de la documentación indicada más arriba.

Para leer los archivos usa un código como el siguiente:

In [ ]:
import json
leer = json.loads(open('Museos.json',encoding="utf8").read())
print(leer)

En los documentos __Estación.csv__ y __Museos.json__ aparece la información de geolocalización en forma de longitud y latitud tanto de las estaciones como de los museos. En este sentido, para calcular la distancia entre dos puntos dadas sus coordenadas se utilizará la siguiente función en Python:

In [2]:
import json
import csv
import math

def haversine(lat1, lon1, lat2, lon2):
    rad=math.pi/180
    dlat=lat2-lat1
    dlon=lon2-lon1
    R=6372.795477598
    a=(math.sin(rad*dlat/2))**2 + math.cos(rad*lat1)*math.cos(rad*lat2)*(math.sin(rad*dlon/2))**2
    distancia=2*R*math.asin(math.sqrt(a))
    return distancia


def leerLocalizacionEstaciones(entrada):
    cabecera = True
    estaciones = []
    for linea in entrada:
        if cabecera:
            cabecera = False
        else:
            #Lista que almacena la informacion de la estacion [codigo,nombre,longitud,latitud]
            estacion = [int(linea[1]),linea[2],float(linea[24]),float(linea[25])]
            estaciones.append(estacion)
    return estaciones

def obtenerEstacionesCercanas(museo, estaciones):
    distancias = []
    for estacion in estaciones:
        distancias.append([estacion[1], haversine(museo[1],museo[2], estacion[3], estacion[2])])
    distancias.sort(key=lambda x:x[1])
    distancias = distancias[0:3]
    return [museo[0],distancias]

csvarchivo = open('Estaciones.csv')
entrada = csv.reader(csvarchivo, delimiter=";")
estaciones = leerLocalizacionEstaciones(entrada)

csvarchivo = open('Salida.csv')
entrada = csv.reader(csvarchivo, delimiter=",")
for linea in entrada:
    print(linea)



leer = json.loads(open('Museos.json',encoding="utf8").read())

for infoMuseo in leer["@graph"]:
    nombreMuseo = infoMuseo["title"]
    latitudMuseo = infoMuseo["location"]["latitude"]
    longitudMuseo = infoMuseo["location"]["longitude"]
    museo = [nombreMuseo, latitudMuseo, longitudMuseo]
    estacionesCercanas = obtenerEstacionesCercanas(museo, estaciones)
    print(estacionesCercanas)
    


['Pza. de España', 'Dióxido de Azufre', '2016.0', 'Monóxido de Carbono', '104.99999999999984', 'Monóxido de Nitrógeno', '6373.0', 'Dióxido de Nitrógeno', '10392.0', 'Partículas <2.5 µm', '0', 'Partículas < 10µm', '0', 'Óxidos de Nitrógeno', '20147.0', 'Ozono', '0', 'Tolueno', '0', 'Benceno', '0', 'Etilbenceno', '0', 'Metaxileno', '0', 'Paraxileno', '0', 'Ortoxileno', '0', 'Hidrocarburos totales(hexano)', '0', 'Metano', '0', 'Hidrocarburos no metánicos (hexano)', '0']
[]
['Escuelas Aguirre', 'Dióxido de Azufre', '2098.0', 'Monóxido de Carbono', '46.700000000000145', 'Monóxido de Nitrógeno', '4819.0', 'Dióxido de Nitrógeno', '12109.0', 'Partículas <2.5 µm', '2595.0', 'Partículas < 10µm', '5230.0', 'Óxidos de Nitrógeno', '19486.0', 'Ozono', '12049.0', 'Tolueno', '632.9000000000001', 'Benceno', '146.8000000000002', 'Etilbenceno', '113.50000000000009', 'Metaxileno', '0', 'Paraxileno', '0', 'Ortoxileno', '0', 'Hidrocarburos totales(hexano)', '375.07000000000016', 'Metano', '352.2100000000001

KeyError: 'location'

#### Normas de entrega

* Fecha tope de entrega: 03/10/2019
* La entrega se realizará subiendo al campus virtual un notebook de Jupyter con la solución. El archivo tendrá como nombre FormatosI_GrupoX donde X será el número de grupo correspondiente.